In [43]:
import tensorflow as tf

import numpy as np
import os
import time

In [44]:
import requests
filename = "sample.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [45]:
# Take a look at the first 250 characters in text
print(raw_text[:250])

there were four of us - george, and william samuel harris, and myself, and
montmorency. we were sitting in my room, smoking, and talking about how bad
we were - bad from a medical point of view i mean, of course.
we were all feeling seedy, and we wer


In [46]:
# The unique characters in the file
vocab = sorted(set(raw_text))
print(f'{len(vocab)} unique characters')

41 unique characters


In [47]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [48]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [49]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[16, 17, 18, 19, 20, 21, 22], [39, 40, 41]]>

In [50]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [51]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [52]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [53]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [54]:
all_ids = ids_from_chars(tf.strings.unicode_split(raw_text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(18789,), dtype=int64, numpy=array([35, 23, 20, ..., 20, 19, 10], dtype=int64)>

In [55]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [56]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

t
h
e
r
e
 
w
e
r
e


In [57]:
seq_length = 100

In [58]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b't' b'h' b'e' b'r' b'e' b' ' b'w' b'e' b'r' b'e' b' ' b'f' b'o' b'u'
 b'r' b' ' b'o' b'f' b' ' b'u' b's' b' ' b'-' b' ' b'g' b'e' b'o' b'r'
 b'g' b'e' b',' b' ' b'a' b'n' b'd' b' ' b'w' b'i' b'l' b'l' b'i' b'a'
 b'm' b' ' b's' b'a' b'm' b'u' b'e' b'l' b' ' b'h' b'a' b'r' b'r' b'i'
 b's' b',' b' ' b'a' b'n' b'd' b' ' b'm' b'y' b's' b'e' b'l' b'f' b','
 b' ' b'a' b'n' b'd' b'\n' b'm' b'o' b'n' b't' b'm' b'o' b'r' b'e' b'n'
 b'c' b'y' b'.' b' ' b'w' b'e' b' ' b'w' b'e' b'r' b'e' b' ' b's' b'i'
 b't' b't' b'i'], shape=(101,), dtype=string)


In [59]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'there were four of us - george, and william samuel harris, and myself, and\nmontmorency. we were sitti'
b'ng in my room, smoking, and talking about how bad\nwe were - bad from a medical point of view i mean, '
b'of course.\nwe were all feeling seedy, and we were getting quite nervous about it. harris\nsaid he felt'
b' such extraordinary fits of giddiness come over him at times, that he\nhardly knew what he was doing; '
b'and then george said that he had fits of giddiness\ntoo, and hardly knew what he was doing. with me, i'


In [60]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [61]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [62]:
dataset = sequences.map(split_input_target)

In [63]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'there were four of us - george, and william samuel harris, and myself, and\nmontmorency. we were sitt'
Target: b'here were four of us - george, and william samuel harris, and myself, and\nmontmorency. we were sitti'


In [64]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [65]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [66]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [67]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [68]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 42) # (batch_size, sequence_length, vocab_size)


In [69]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  10752     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  43050     
                                                                 
Total params: 3,992,106
Trainable params: 3,992,106
Non-trainable params: 0
_________________________________________________________________


In [70]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [71]:
sampled_indices

array([40, 34, 18,  3, 36, 14, 14, 30,  2, 17,  5, 19, 25,  7, 41, 25, 27,
       11, 17, 34, 16,  7,  9, 28, 14,  7, 35, 15, 28, 35, 21, 33,  5,  1,
        0, 21,  7, 35, 14,  2,  9, 39, 32,  7, 37, 22, 17, 38, 12,  3, 30,
       34, 34,  3, 38,  7, 11,  4, 38, 16, 33, 19,  5,  9, 29,  7, 32, 22,
       23, 23, 34, 20,  8,  9, 14, 30,  4, 14, 16, 41, 35, 14, 20, 30, 39,
        1, 23,  0, 38, 38, 10,  6, 19, 33, 11, 10,  7,  4, 30, 31],
      dtype=int64)

In [72]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'rp every night. and don\'t stuff up your head with things you\ndon\'t understand."\ni followed the direc'

Next Char Predictions:
 b'ysc!u;;o b\'dj)zjl1bsa)-m;)t?mtfr\'\n[UNK]f)t; -xq)vgbw6!oss!w)1"ward\'-n)qghhse,-;o";azt;eox\nh[UNK]ww.(dr1.)"op'


In [73]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [74]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 42)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.7383184, shape=(), dtype=float32)


In [75]:
tf.exp(example_batch_mean_loss).numpy()

42.02726

In [76]:
model.compile(optimizer='adam', loss=loss)

In [77]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [78]:
EPOCHS = 20

In [79]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
2/2 [==============================] - 7s 2s/step - loss: 3.7198
Epoch 2/20
2/2 [==============================] - 5s 2s/step - loss: 3.5697
Epoch 3/20
2/2 [==============================] - 5s 2s/step - loss: 4.7335
Epoch 4/20
2/2 [==============================] - 5s 2s/step - loss: 3.5233
Epoch 5/20
2/2 [==============================] - 5s 2s/step - loss: 3.5701
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 3.5697
Epoch 7/20
2/2 [==============================] - 4s 2s/step - loss: 3.5508
Epoch 8/20
2/2 [==============================] - 4s 2s/step - loss: 3.5198
Epoch 9/20
2/2 [==============================] - 4s 2s/step - loss: 3.4672
Epoch 10/20
2/2 [==============================] - 4s 2s/step - loss: 3.3870
Epoch 11/20
2/2 [==============================] - 4s 2s/step - loss: 3.2569
Epoch 12/20
2/2 [==============================] - 5s 2s/step - loss: 3.1039
Epoch 13/20
2/2 [==============================] - 5s 2s/step - loss: 3.0279
Epoch 14

In [80]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [81]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [83]:
start = time.time()
states = None
next_char = tf.constant(['GEORGE:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

GEORGE:lhss wmadk iushnd"dewno!ked i, te hhh aeo trhe aad hrt ee anisel ol hay chn t aer n ta o,fawha awoo oy se zysf nua ito ouom ihcant a,dde orhled nt toot hnprodeut cewheen tes
mgbrehaedt utt ah iinaese
hcsee ihoi puiue
cfl,
atyv nogiey veni'wiinsteed auhi tle utheo s tlhelhwere ao eead omg tre ted e eh, aees oit , ie
ne sa tschhlit hebn hhe eeo ns hnindt morkokhed as tho telne eho
n cee ngni lee mcfesrhialsig oe ltyte ui xthe g temole ipaw as rilonelte ohde wie net ie toaadye asphtifdshati ld theem assee io dd
e a,eshe,
uany th iasre dt hrsun id mea heorcmkpe adcd gf thceewgine ebndmlwbtehemouhtaeroe woeleualen-i ma ot e.t co ee h uetny te te t hhe
 ou sokulwrtrmo,y m hnwteb, me ntho ma tdheoeees sise, ee otiee e oaft tte o ue d  er se t tit hei goce wat d hai asa  ltt hsednfe tsed ce ithfw ouetd gidsn,t heo leth satorr e mnn hgwt w e at a dtechceerpheey wee s e t;e  auos iou  ontoo thtotetr odne lod o s addaui,e eofr laeedt hhoi nege, sheny
tvn toi ethottdteased noa ednot man me 